In [ ]:
# 📦 INSTALLATION
!pip install flashtext openpyxl
!pip install pycountry
!pip install country_capitals

  Preparing metadata (setup.py) ... done
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9300 sha256=bde7405217c7837ff3fd0e08c952e4b7b38f5aaf1407988f2b67f85552fd7c76
  Stored in directory: /root/.cache/pip/wheels/49/20/47/f03dfa8a7239c54cbc44ff7389eefbf888d2c1873edaaec888
Successfully built flashtext


In [ ]:
import pycountry
import unicodedata
from babel import Locale
from country_capitals import get_capital

def normalize_string(text):
    """Normalise les chaînes de caractères pour enlever les accents et caractères spéciaux"""
    return unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')

def create_country_dictionary():
    country_dict = {}
    locale_fr = Locale('fr')  # Pour les noms en français

    for country in pycountry.countries:
        try:
            # Nom anglais
            name = country.name

            # Nom officiel
            official_name = getattr(country, 'official_name', name)

            # Code ISO3
            iso3 = country.alpha_3

            # Capitale
            capital = get_capital(name)

            # Nom français via babel
            name_fr = locale_fr.territories.get(country.alpha_2)

            # Création de l'entrée dans le dictionnaire
            country_dict[normalize_string(name).upper()] = [
                name,           # Nom en anglais
                official_name,  # Nom officiel
                iso3,           # Code ISO3
                capital,        # Capitale
                name_fr         # ✅ Nom en français
            ]

        except Exception as e:
            print(f"Erreur pour le pays {country}: {str(e)}")
            continue

    return country_dict

# Création du dictionnaire complet
country_dictionary = create_country_dictionary()

Erreur pour le pays Country(alpha_2='AW', alpha_3='ABW', flag='🇦🇼', name='Aruba', numeric='533'): Could not find a capital for Aruba.
Erreur pour le pays Country(alpha_2='AI', alpha_3='AIA', flag='🇦🇮', name='Anguilla', numeric='660'): Could not find a capital for Anguilla.
Erreur pour le pays Country(alpha_2='AX', alpha_3='ALA', flag='🇦🇽', name='Åland Islands', numeric='248'): Could not find a capital for Åland Islands.
Erreur pour le pays Country(alpha_2='AS', alpha_3='ASM', flag='🇦🇸', name='American Samoa', numeric='016'): Could not find a capital for American Samoa.
Erreur pour le pays Country(alpha_2='AQ', alpha_3='ATA', flag='🇦🇶', name='Antarctica', numeric='010'): Could not find a capital for Antarctica.
Erreur pour le pays Country(alpha_2='TF', alpha_3='ATF', flag='🇹🇫', name='French Southern Territories', numeric='260'): Could not find a capital for French Southern Territories.
Erreur pour le pays Country(alpha_2='BQ', alpha_3='BES', flag='🇧🇶', name='Bonaire, Sint Eustatius and 

In [ ]:
country_dictionary['COMOROS']

['Comoros', 'Union of the Comoros', 'COM', 'Moroni', 'Comores']

In [ ]:
# 📚 IMPORTS
import pandas as pd
import os
import zipfile
from collections import defaultdict
from io import BytesIO
from flashtext import KeywordProcessor
from tqdm import tqdm

def upload_file(prompt):
    """Helper function to handle file uploads"""
    print(prompt)
    try:
        from google.colab import files
        uploaded = files.upload()
        return list(uploaded.keys())[0]
    except ImportError:
        raise ImportError("Ce script est prévu pour un environnement Google Colab.")

def process_file(filepath, archive, kp):
    """Process individual Excel file"""
    filename = os.path.basename(filepath)
    try:
        df = pd.read_excel(filepath, engine='openpyxl')

        # Vérification des colonnes d'affiliation
        cols_lower = [col.strip().lower() for col in df.columns]
        aff_col = None

        if any("adresse (ad)" in col.lower() for col in df.columns):
            for col in df.columns:
                if "adresse (ad)" in col.lower():
                    aff_col = col
                    break
            df.rename(columns={aff_col: 'Affiliation'}, inplace=True)
            df["Affiliation"] = df["Affiliation"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip().str.lower()

        elif 'affiliation' in cols_lower:
            aff_col = df.columns[cols_lower.index('affiliation')]
        else:
            print(f"❌ Ni 'Affiliation' ni 'Adresse (AD)' trouvées dans {filename}, fichier ignoré.")
            return None

        if 'PMID' not in df.columns:
            df['PMID'] = range(1, len(df) + 1)

        # Détection des pays
        def detecter_pays_flashtext(texte):
            if pd.isna(texte):
                return set()
            keywords_found = kp.extract_keywords(str(texte))
            return set(keywords_found) if keywords_found else set()

        df["Pays_detectés"] = df["Affiliation"].apply(detecter_pays_flashtext)
        df["Pays_detectés"] = df["Pays_detectés"].apply(
            lambda s: "; ".join(sorted(s)) if s else ""
        )
        df["country_of_authors"] = df["Pays_detectés"]

        # Enregistrement temporaire
        buffer = BytesIO()
        with pd.ExcelWriter(buffer, engine='openpyxl') as writer:
            df.to_excel(writer, index=False)

        archive.writestr(filename, buffer.getvalue())
        return f"✅ Traité et ajouté au ZIP : {filename}"

    except Exception as e:
        return f"❌ Erreur lors du traitement de {filename}: {str(e)}"

# --- EXÉCUTION PRINCIPALE ---
try:
    # ⬆️ Upload ZIP contenant les fichiers Excel
    zip_filename = upload_file("➡️ Téléversez le fichier ZIP contenant les fichiers .xlsx…")

    # 📂 EXTRACTION DES FICHIERS
    INPUT_DIR = "extracted_files"
    OUTPUT_DIR = "fichier_des_pays"
    ZIP_OUTPUT_PATH = "resultats_pays_detectes.zip"

    os.makedirs(INPUT_DIR, exist_ok=True)
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(INPUT_DIR)

    # 🔍 Fichiers Excel à traiter
    excel_files = [
        os.path.join(root, file)
        for root, _, files_ in os.walk(INPUT_DIR)
        for file in files_
        if file.lower().endswith((".xlsx", ".xls"))
    ]
    if not excel_files:
        raise FileNotFoundError("❌ Aucun fichier Excel trouvé dans le dossier.")

    # 🔍 Création du dictionnaire de variantes à partir de country_dictionary
    country_variants = defaultdict(set)
    for fr_name, values in country_dictionary.items():
        name_en = values[0]
        official_name = values[1]
        name_fr = values[4]

        variants = set()
        for val in [name_en, official_name, name_fr]:
            if val:
                variants.update({
                    val.lower(),
                    val.upper(),
                    val.title()
                })

        country_variants[name_fr or name_en].update(variants)

    # ⚙️ Initialisation de FlashText
    kp = KeywordProcessor(case_sensitive=False)
    for country, variants in country_variants.items():
        for variant in variants:
            kp.add_keyword(variant, country)

    # 📦 Création du dossier output
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # ▶️ Traitement de chaque fichier Excel
    with zipfile.ZipFile(ZIP_OUTPUT_PATH, mode='w', compression=zipfile.ZIP_DEFLATED) as archive:
        for filepath in tqdm(excel_files, desc="🔄 Traitement des fichiers", total=len(excel_files)):
            result = process_file(filepath, archive, kp)
            if result:
                print(result)

    print(f"\n🎉 Tous les fichiers ont été traités et archivés dans : {ZIP_OUTPUT_PATH}")

    # ⬇️ Téléchargement Colab
    try:
        from google.colab import files
        files.download(ZIP_OUTPUT_PATH)
    except ImportError:
        print(f"Fichier disponible ici : {os.path.abspath(ZIP_OUTPUT_PATH)}")

except Exception as e:
    print(f"Une erreur est survenue : {str(e)}")


➡️ Téléversez le fichier ZIP contenant les fichiers .xlsx…


Saving resultats_entites.zip to resultats_entites (1).zip


🔄 Traitement des fichiers:   2%|▏         | 1/54 [00:01<00:57,  1.08s/it]

✅ Traité et ajouté au ZIP : Ivory Coast_entites.xlsx


🔄 Traitement des fichiers:   4%|▎         | 2/54 [00:02<00:53,  1.04s/it]

...

✅ Traité et ajouté au ZIP : Namibia_entites.xlsx


🔄 Traitement des fichiers: 100%|██████████| 54/54 [01:27<00:00,  1.61s/it]

✅ Traité et ajouté au ZIP : Congo_entites.xlsx

🎉 Tous les fichiers ont été traités et archivés dans : resultats_pays_detectes.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>